In [ ]:
#!pip3 install cartesian_viz

In [ ]:
from IPython.core.display import display, HTML

#%matplotlib inline
%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [ ]:
import os
import sys
import numpy as np
from scipy.spatial.transform import Rotation as R
import numpy as np

import gtsam
from gtsam import Point3, Pose3, Rot3

from cartesian_viz.engine import BaseVisualiser
from cartesian_viz.draw_descriptors import PointDesc, Pose2DDrawDesc, UncertainPose2DDrawDesc


In [ ]:
SRC_DIR = os.path.abspath(f'./src')
sys.path.append(SRC_DIR)
SRC_DIR

In [ ]:
from transform import Transform
from uncertain_transform import UncertainTransform

### Utility functions for visualization

In [ ]:
import angles

def get_world_yaw_from_transform(transform):
    dz = transform.rotation_matrix[:,2]
    angle = -np.rad2deg(np.arctan2(dz[0], dz[2]))
    return angles.normalize(angle, 0, 360)
    
    
def get_wedge_width_from_cov(covariance_matrix):
    line_width = np.degrees(np.sqrt(covariance_matrix[1,1]))
    line_width = 1 if line_width < 1 or np.isnan(line_width) else line_width
    return line_width


def get_viz_data_from_transform(uncertain_transform, color, show_errors, circle_radius, transform_text=None,ellipse_color=None):
    x, _, z = uncertain_transform.position
    heading = get_world_yaw_from_transform(uncertain_transform)
    line_width, e_angle, e_x, e_z = 20, 0, 0.01, 0.01
    if show_errors:
            e_angle, e_x, e_z = uncertain_transform.compute_ellipse_from_covariance()
            e_angle = np.deg2rad(e_angle)
            line_width = get_wedge_width_from_cov(uncertain_transform.covariance_matrix)

    return  {
            "latitude":z, "longitude":x, "facing":heading,
            "facing_error":line_width, "ellipse_angle":e_angle, "size_x":e_x, "size_y":e_z,
            "color":color,
            "point_radius":circle_radius,
            "ellipse_color": ellipse_color if ellipse_color else color,
            }


def get_figure():
    bviz = BaseVisualiser(relative_frame=True)
    bviz.add_object_desc(PointDesc(radius=0.2, alpha=0.3))
    bviz.add_object_desc(Pose2DDrawDesc())
    bviz.add_object_desc(UncertainPose2DDrawDesc(radius=0.1))
    return bviz

def show_transform(transform, base_viz, transform_text=None, circle_color = None, circle_radius = 0.5, ellipse_color=None):
    """Draws the object represented by `transform` on figure p at a given latlon (if provided), or at a relative position."""

    if circle_color is None:
        circle_color = ellipse_color if ellipse_color else 'red' 
        
    has_err =  hasattr(transform, "covariance_matrix")
    entry = get_viz_data_from_transform(transform, circle_color, has_err, circle_radius, transform_text=transform_text, ellipse_color=ellipse_color )
    if not has_err:
        base_viz.add_entry_object(Pose2DDrawDesc().get_name(), entry)
    else:
        base_viz.add_entry_object(UncertainPose2DDrawDesc().get_name(), entry)

def show_ms_transform(uncertain_transform, base_viz, transform_text=None, circle_color = None, circle_radius = 0.5, ellipse_color=None, draws=500):
    samples = uncertain_transform.get_samples(nb_samples=draws)

    c = ellipse_color if ellipse_color else (circle_color if circle_color else 'red')
    for s in samples:
        entry = get_viz_data_from_transform(s, c, False, circle_radius/2)
        base_viz.add_entry_object(PointDesc().get_name(), entry)

    show_transform(uncertain_transform, base_viz, transform_text=transform_text, circle_color=circle_color, circle_radius=circle_radius, ellipse_color=ellipse_color)



def create_measurment_from_depth_prediction(position, error_rate_per_mt=0.01):
    """
    Computes a uncertain transform from the predocton positions
    :param position: detection 3D position
    :param error_per_m: the rate of error increse per meter
    """
    distance = np.linalg.norm(position)
    pdir = position/distance
    
    m = np.arccos(pdir.dot(np.array([0,0,1])))
    orientation = R.from_rotvec(np.array([0,-m,0])).as_quat()    
    depth_error = 0.1 + distance * error_rate_per_mt
    other_axis = 1e-1
    error_std = np.array((other_axis,other_axis,  depth_error))
    orientaion_error = np.radians(np.ones(3)*0.1) #unknown orientation

    # construct covariance matrix
    covariance_matrix = np.identity(6)
    covariance_matrix[:3,:3]=covariance_matrix[:3,:3] * np.power(orientaion_error.T,2)
    covariance_matrix[3:,3:]=covariance_matrix[3:,3:] * np.power(error_std.T,2)
    return UncertainTransform(position, orientation, covariance_matrix=covariance_matrix)



## Merge measurements

In [ ]:

trip_figure = get_figure()

transform = Transform.fromEuler((1,0,1),(33,0,0))

m1 = UncertainTransform.fromEuler((0.1,0,0),(104,0,0),(0.5,0.01,1.75),(0,11.5,0)).apply_transform(transform)
m2 = UncertainTransform.fromEuler((0.1,0,0),(144,0,0),(0.8,0.01,1.75),(0,5,0)).apply_transform(transform)
m3 = UncertainTransform.fromEuler((0.1,0,0),(33,0,0),(0.6,0.1,1.3),(0,11,0)).apply_transform(transform)

m4 = UncertainTransform.merge_measurments([m1,m2])

show_transform(m1, trip_figure, circle_radius=0.2, ellipse_color="green", circle_color="green")
show_transform(m2, trip_figure, circle_radius=0.2, ellipse_color="yellow", circle_color="yellow")
#show_transform(m3, trip_figure, circle_radius=0.2, ellipse_color="blue", circle_color="blue")
show_transform(m4, trip_figure, circle_radius=0.2, ellipse_color="red")
trip_figure.display()


## Pose error composition

In [ ]:
m1 = create_measurment_from_depth_prediction([5,0,5], error_rate_per_mt=0.01)


trip_figure = get_figure()
show_ms_transform(m1, trip_figure, circle_radius=0.01, draws=40)
trip_figure.display()


In [ ]:
trip_figure = get_figure()
pos = (20,0,10)
rot = (5,0,0)
m2 = UncertainTransform.fromEuler(pos,rot,(0.2,0.01,.01),(0,1,0))
show_ms_transform(m2,  trip_figure, circle_radius=0.1,ellipse_color="blue",draws=5)

m3 = m1.apply_transform(m2)
show_ms_transform(m3,  trip_figure, circle_radius=0.2,ellipse_color="green", draws=300)


m2z = UncertainTransform.fromEuler(pos,rot,(0,0,0),(0,0.001,0))
m3z = m1.apply_transform(m2z)
show_ms_transform(m3z, trip_figure, circle_radius=0.1, draws=10)


trip_figure.display()



### Show effect of rotation error from camera to position of the object

In [ ]:
c2w_p = np.array([4,0,0])
c2w_o = np.array([-0,0,0])
obj2c = Transform.fromEuler([20,0,20],[62,0,0])

p_err_std = np.array([.5,0,.5])
o_err_std = np.array([10,0,0])


trip_figure = get_figure()

draws = 1000
c2w = Transform.fromEuler(c2w_p,c2w_o)

c2w_measured = UncertainTransform.fromEuler(c2w_p,c2w_o, p_err_std, o_err_std[[1,0,2]])
show_ms_transform(c2w_measured, trip_figure, circle_radius=0.4, ellipse_color="red", circle_color="blue")

obj2w_measured = UncertainTransform(obj2c.position,obj2c.rotation_quat, np.ones((6,6)) * 1e-4).apply_transform(c2w_measured)#.apply_transform(c2w_measured).apply_transform(c2w_measured))
print(c2w_measured)
show_ms_transform(obj2w_measured, trip_figure, circle_radius=0.4, ellipse_color="green", circle_color="red")

trip_figure.display()


## Relative pose simple transform

In [ ]:
T_aw = Transform.fromEuler((1.4,0,3),(145,0,0))
T_bw = Transform.fromEuler((1,4,0),(-35,0,0))
T_ab = T_aw.between(T_bw)




trip_figure = get_figure()
show_transform(T_aw, trip_figure, circle_radius=0.4, ellipse_color="green", circle_color="green")
show_transform(T_bw, trip_figure, circle_radius=0.4, ellipse_color="yellow", circle_color="yellow")
show_transform(T_ab.apply_transform(T_bw), trip_figure, circle_radius=0.2, ellipse_color="red", circle_color="red")
trip_figure.display()






## Relative pose measured transform

In [ ]:
T_aw = UncertainTransform.fromEuler((1,4,0),(-15,0,0), (4,0,0.1),(0,3,0))
T_bw = UncertainTransform.fromEuler((1.4,0,3),(75,0,0), (0.5,0,0.5),(0,3,0))
T_ab = T_bw.between(T_aw)




trip_figure = get_figure()
show_ms_transform(T_aw, trip_figure, circle_radius=0.1, ellipse_color="green", circle_color="green")
show_ms_transform(T_bw, trip_figure, circle_radius=0.1, ellipse_color="yellow", circle_color="yellow")
show_transform(T_ab, trip_figure, circle_radius=0.1, ellipse_color="red", circle_color="red")
trip_figure.display()



